# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [2]:
import numpy as np
import os
import pickle
import six.moves.urllib as urllib
import sys
sys.path.append("..")
import tarfile
import tensorflow as tf
import zipfile
from object_detection.eval_util import evaluate_detection_results_pascal_voc

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline
%load_ext autoreload
%autoreload 2
# This is needed since the notebook is stored in the object_detection folder.

from utils import label_map_util

from utils import visualization_utils as vis_util

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util
def get_annotations(image_path):
    img_id = os.path.basename(image_path)[:-4]
    annotation_path = os.path.join(
    os.path.split(os.path.dirname(image_path))[0], 'Annotations',
    '{}.xml'.format(img_id)
    )
    return xml_to_dict(annotation_path)
from utils.kitti import show_groundtruth, create_results_list
from utils.kitti import visualize_predictions
import glob

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [54]:
# What model to download.
FREEZE_DIR = 'atrous_frozen_v2/'
PATH_TO_CKPT = os.path.join(FREEZE_DIR,
                            'frozen_inference_graph.pb'
                           )
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'kitti_map.pbtxt')

NUM_CLASSES = 9

## Load a (frozen) Tensorflow model into memory.

In [55]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [56]:
PATH_TO_LABELS = os.path.join('data', 'kitti_map.pbtxt')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, 
                                                            max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [58]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [59]:
with open('kitti_data/train.txt') as f:
    train_ids = f.readlines()[0].split(',')
with open('kitti_data/valid.txt') as f:
    valid_ids = f.readlines()[0].split(',')

In [60]:
len(train_ids)

6732

In [61]:
len(valid_ids)

749

# Detection

In [62]:
PATH_TO_TEST_IMAGES_DIR = 'voc_kitti_valid/VOC2012/JPEGImages/'
p = 'voc_kitti_valid/VOC2012/JPEGImages/1023.jpg'
TEST_IMAGE_PATHS = [ p]
FIGSIZE = (20, 20)

In [63]:
import glob
def glob_base(pat): return list(map(os.path.basename, glob.glob(pat)))

In [65]:
from create_dataset import *

### Check that valid files dont overlap with train files

In [67]:
valid_ids = glob_base(VOC_VALID_DIR + '/VOC2012/JPEGImages/*.jpg')
train_ids = glob_base(VOC_TRAIN_DIR+ '/VOC2012/JPEGImages/*.jpg')

assert len(pd.Index(valid_ids).intersection(train_ids)) == 0

In [68]:
test_dir = 'voc_kitti_valid/VOC2012/JPEGImages/'
test_image_paths = [os.path.join(test_dir, x) for x in valid_ids]

In [69]:
len(test_image_paths)

749

In [70]:
train_labs= glob.glob('kitti_data/training/label_2/*.txt')
test_labs = glob.glob('kitti_data/valid/label_2/*.txt')

### Calculate MaP by category (8 mins roughly)

In [73]:
%%time
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        res = create_results_list(test_image_paths, sess, detection_graph)

CPU times: user 5min 29s, sys: 0 ns, total: 5min 29s
Wall time: 8min 14s


In [74]:
import pandas as pd
perf = pd.Series(evaluate_detection_results_pascal_voc(res, categories))

../object_detection/utils/metrics.py:145: RuntimeWarning: invalid value encountered in true_divide
  num_images_correctly_detected_per_class / num_gt_imgs_per_class)


In [75]:
perf

PerformanceByCategory/mAP@0.5IOU/car               0.967502
PerformanceByCategory/mAP@0.5IOU/cyclist           0.887509
PerformanceByCategory/mAP@0.5IOU/dontcare          0.444671
PerformanceByCategory/mAP@0.5IOU/misc              0.867442
PerformanceByCategory/mAP@0.5IOU/pedestrian        0.821766
PerformanceByCategory/mAP@0.5IOU/person_sitting    0.776042
PerformanceByCategory/mAP@0.5IOU/tram              0.980379
PerformanceByCategory/mAP@0.5IOU/truck             0.956242
PerformanceByCategory/mAP@0.5IOU/van               0.964281
Precision/mAP@0.5IOU                               0.851759
dtype: float64

### Make nice performance table

In [76]:
def clean_idx(perf):
    x = list(perf.index.map(lambda x: x[33:]))
    x[-1] = 'Total'
    perf.index = x
    return perf

In [84]:
perf = clean_idx(perf)

In [85]:
perf.to_frame('rcnn_mAP')#.round(3).to_csv('~/Desktop/faster_rcnn_mAP_by_category.csv')

,rcnn_mAP
car,0.967502
cyclist,0.887509
dontcare,0.444671
misc,0.867442
pedestrian,0.821766
person_sitting,0.776042
tram,0.980379
truck,0.956242
van,0.964281
Total,0.851759


In [77]:
def get_dict_slice(res, slc_obj):
    '''get a slice of the values for each key in a dict'''
    output = {}
    for k in res.keys():
        output[k] = res[k][slc_obj]
    return output
        
        

### Calculate MaP for each image

In [78]:
%%capture
img_scores = {image_id: evaluate_detection_results_pascal_voc(
    get_dict_slice(res, slice(i, i+1)), categories)
              for i, image_id in enumerate((res['image_id'][:-1]))}
                                           
                                           

In [80]:
OVERALL_PERF_KEY = 'Precision/mAP@0.5IOU'

In [81]:
#pickle.dump(res, open('mobile_net_valid_results_dct.pkl', 'wb'))

In [82]:
res['image_id'][0]

'3528'

In [91]:
from kitti_constants import name_to_id
from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array
from utils.kitti import get_boxes_scores_classes, visualize_predictions

In [48]:
def get_img_scores(image_path):
    imageid = os.path.basename(image_path)[:-4]
    return pd.Series(img_scores[imageid]).round(2).dropna()
        

In [ ]:
%%time
%precision 4
import time
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_path = np.random.choice(test_image_paths)
        image = Image.open(image_path)
        image_np = load_image_into_numpy_array(image)
        start = time.time()
        image_process = visualize_predictions(image_np, sess, detection_graph)
        # boxes, scores, classes, num_detections = get_boxes_scores_classes(image_np, sess, detection_graph)
        print('inference time: {} seconds'.format(
            np.round(time.time() -  start, 2)))
        print ('MaP scores\n{}'.format(get_img_scores(image_path)))
        plt.figure(figsize=FIGSIZE)
        plt.imshow(image_process)
        plt.title('Model', fontsize=16)
        #plt.imsave(image_process, 'worst_prediction labs.jpg')
        plt.figure(figsize=FIGSIZE)
        truth_img = show_groundtruth(image_path)
        plt.imshow(truth_img)
        plt.title('Human Labels', fontsize=16)
        plt.figure(figsize=FIGSIZE)
        plt.imshow(load_image_into_numpy_array(Image.open(image_path)))
        plt.title('Raw Image')
        #plt.savefig('worst_prediction labs.jpg')